In [1]:
from sklearn.metrics.pairwise import pairwise_distances
from tensorflow.keras.utils import multi_gpu_model
from model import identity_block, conv_block
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import layers
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [3]:
lr=1e-4
epochs=50
decay=1e-2
num_classes=35

data_dir="data/CSI_preprocessed_35.h5"
intruder_dir="data/CSI_preprocessed_Intruder.h5"

In [3]:
hf = h5py.File(data_dir, 'r')
X_train = np.expand_dims(hf.get('X_train'), axis=-1)
X_test = np.expand_dims(hf.get('X_test'), axis=-1)
y_train = np.array(hf.get('y_train'))
y_test = np.array(hf.get('y_test'))
y_train_onehot = np.eye(num_classes)[hf.get('y_train')]
y_test_onehot = np.eye(num_classes)[hf.get('y_test')]
hf.close()

hf = h5py.File(intruder_dir, 'r')
X_data = np.expand_dims(hf.get('X_data'), axis=-1)
hf.close()

print(X_train.shape, X_data.shape)

(1189, 500, 270, 1) (200, 500, 270, 1)


In [4]:
input_layer = layers.Input(shape=(X_train.shape[1:]))

x = layers.Conv2D(64, (7, 7), strides=(2, 2))(input_layer)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, [64, 256], "relu")
x = identity_block(x, [64, 256], "relu")

x = conv_block(x, [128, 512], "relu")
x = identity_block(x, [128, 512], "relu")

x = conv_block(x, [256, 1024], "relu")
x = identity_block(x, [256, 1024], "relu")

x = conv_block(x, [512, 2048], "relu")
x = identity_block(x, [512, 2048], "relu")

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(num_classes, activation=None)(x)
x = layers.Activation('softmax')(x)

model_base = Model(inputs=input_layer, outputs=x)
model_base.summary()
model = multi_gpu_model(model_base, gpus=3)

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr, decay=decay), metrics=['acc'])
history = model.fit(x=X_train, y=y_train_onehot, epochs=epochs, validation_data=(X_test, y_test_onehot), verbose=2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500, 270, 1)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 247, 132, 64) 3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 247, 132, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 247, 132, 64) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
max_poolin

In [5]:
model_evm = Model(inputs=model_base.get_layer("input_1").input , outputs=model_base.get_layer("dense").output)

In [6]:
X_evm_train = model_evm.predict(X_train)
X_evm_test = model_evm.predict(X_test)
X_evm_data = model_evm.predict(X_data)

In [7]:
print(X_evm_train.shape)

(1189, 35)


In [8]:
#https://github.com/mlosch/python-weibullfit
    def fit(x, iters=1000, eps=1e-6):
    """
    Fits a 2-parameter Weibull distribution to the given data using maximum-likelihood estimation.
    :param x: 1d-ndarray of samples from an (unknown) distribution. Each value must satisfy x > 0.
    :param iters: Maximum number of iterations
    :param eps: Stopping criterion. Fit is stopped ff the change within two iterations is smaller than eps.
    :return: Tuple (Shape, Scale) which can be (NaN, NaN) if a fit is impossible.
        Impossible fits may be due to 0-values in x.
    """
    # fit k via MLE
    ln_x = np.log(x+eps)
    k = 1.
    k_t_1 = k

    for t in range(iters):
        x_k = x ** k
        x_k_ln_x = x_k * ln_x
        ff = np.sum(x_k_ln_x)
        fg = np.sum(x_k)
        f = ff / fg - np.mean(ln_x) - (1. / k)

        # Calculate second derivative d^2f/dk^2
        ff_prime = np.sum(x_k_ln_x * ln_x)
        fg_prime = ff
        f_prime = (ff_prime/fg - (ff/fg * fg_prime/fg)) + (1. / (k*k))

        # Newton-Raphson method k = k - f(k;x)/f'(k;x)
        k -= f/f_prime

        if np.isnan(f):
            return np.nan, np.nan
        if abs(k - k_t_1) < eps:
            break

        k_t_1 = k

    lam = np.mean(x ** k) ** (1.0 / k)

    return k, lam


def psi_i_dist(dist, k_i, lambda_i):
    """
    Gives the probability of sample inclusion
    :param dist: Numpy vector of distances between samples
    :param lambda_i: Scale of the Weibull fitting
    :param k_i: Shape of the Weibull fitting
    :return: PSI = Probability of Sample Inclusion. This is the probability that x' is included in the boundary estimated by x_i
    """
    return np.exp(-(((np.abs(dist))/lambda_i)**k_i))

In [9]:
# fit weibull models
tow = 50
weibull_models = []
for i in range(num_classes):
    train_ind = np.squeeze(np.where(y_train == i))
    D = pairwise_distances(X_evm_train[train_ind], X_evm_train, metric="euclidean", n_jobs=1)
    for j in range(D.shape[0]):
        D_tmp = np.sort(D[j])
        D_tmp = D_tmp[np.where(D_tmp>0)][:tow]
        weibull_models.append(fit(D_tmp, iters=100, eps=1e-6))
weibull_models = np.array(weibull_models)
weibull_models.shape

(1189, 2)

In [10]:
# get extreme value machine perdictions
def get_evm(data):
    D = pairwise_distances(X_evm_train, data, metric="euclidean", n_jobs=1)
    preds = np.zeros_like(D)
    for i in range(X_evm_train.shape[0]):
        for j in range(data.shape[0]):
            preds[i, j] = psi_i_dist(D[i, j], weibull_models[i][0], weibull_models[i][1])
    return preds

In [11]:
evm = get_evm(X_evm_test)
np.mean(np.equal(y_train[np.argmax(evm, axis=0)], y_test))*100

99.04761904761905